# 学习Tensorflow


## 1. tf.Variable[来自Tensorflow Programmer Guide]
当你要训练一个模型时，你用variables来保持（hold）并更新模型参数。Variables是in-memory（内存？）中包含张量的缓存。它们必须被显式地初始化，训练结束后可以被存储到磁盘中。
### 1. 1 创建
当你创建一个Variable时，你将一个tensor作为初始值传递给Variable()的constructor。
记住，所有创建的操作都会要求你制定tensor的形状（shape），这个shape会自动变成variable的shape

In [ ]:
import tensorflow as tf
# Create two variables
weights = tf.Variable(tf.random_normal([784,200],stddev=0.35),name="weights")
biases = tf.Variable(tf.zeros([200]),name="biases")
print weights

tf.Variable()添加几个操作到图(graph)中:
- 一个 variable 来保存变量值
- 一个初始化操作，将变量设置为初始值
- 添加初始值的操作，例如上面的zeros()操作也被添加到图中

返回值是tf.Variable类

### 1.2 初始化
在你的模型可以被运行之前，variable的initializer必须要先执行。最简单的办法是添加一个操纵，让所有的variable initialer都跑起来。

In [ ]:
# add an op to initialize the variables
init_op = tf.global_variables_initializer()

# Later, when launching the model
with tf.Session() as sess:
    #Run the init op
    sess.run(init_op)
    
    #Use the model

#### 1.2.1 从另一个Variable初始化

In [ ]:
# create another variable with the same value as 'weights'
w2 = tf.Variable(weights.initialized_value(), name='w2')

## 1.2 Saving and Restoring
最简单save and restore一个模型是适用 `tf.train.Saver`.

### Checkpoint Files
Variables以二进制形式被存储起来，大体上是variable名字和张量值的映射。
要理解什么variable存储在checkpoint中，你可以使用inspect_checkpoint库，调用`print_tensors_in_checkpoint_file`函数。

In [ ]:
# Create some variables.
v1 = tf.Variable(..., name="v1")
v2 = tf.Variable(..., name="v2")
...
# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, save the
# variables to disk.
with tf.Session() as sess:
  sess.run(init_op)
  # Do some work with the model.
  ..
  # Save the variables to disk.
  save_path = saver.save(sess, "/tmp/model.ckpt")
  print("Model saved in file: %s" % save_path)
    
    
# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "/tmp/model.ckpt")
  print("Model restored.")
  # Do some work with the model

### 选择哪个Variable来Save and Restore
如果你不传给`tf.train.Saver()`任何参数，那它将会存储图中所有的variables。
有时候你可能训练模型时候给variable其一个名字，例如‘weights’，但是save时候想换成另一个名字，例如‘params’。或者有时候你只想保存一个模型的一部分variables。你可以很简单地指定要存储的名字和variable，传给`tf.train.Saver()`。
- 你可以创建多个saver对象
- 如果你在一个session开头，只想restore一部分模型的variables，那剩下其他的variables你要记得初始化。

In [ ]:
# Create some variables.
v1 = tf.Variable(..., name="v1")
v2 = tf.Variable(..., name="v2")
...
# Add ops to save and restore only 'v2' using the name "my_v2"
saver = tf.train.Saver({"my_v2": v2})
# Use the saver object normally after that.